In [2]:
import math

In [3]:
((2/5)*math.log(5/2,2)) + ((3/5)*math.log(5/3,2))

0.9709505944546687

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [85]:
horse= pd.read_csv(r'C:\Users\excel\OneDrive\Desktop\horse.csv')

In [86]:
horse.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,45.0,8.4,NaN,NaN,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,NaN,NaN,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,74.0,7.4,NaN,NaN,died,no,4300,0,0,no


In [87]:
horse.shape

(299, 28)

In [88]:
horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                299 non-null    object 
 1   age                    299 non-null    object 
 2   hospital_number        299 non-null    int64  
 3   rectal_temp            239 non-null    float64
 4   pulse                  275 non-null    float64
 5   respiratory_rate       241 non-null    float64
 6   temp_of_extremities    243 non-null    object 
 7   peripheral_pulse       230 non-null    object 
 8   mucous_membrane        252 non-null    object 
 9   capillary_refill_time  267 non-null    object 
 10  pain                   244 non-null    object 
 11  peristalsis            255 non-null    object 
 12  abdominal_distention   243 non-null    object 
 13  nasogastric_tube       195 non-null    object 
 14  nasogastric_reflux     193 non-null    object 
 15  nasoga

In [89]:
horse.isnull().sum()

surgery                    0
age                        0
hospital_number            0
rectal_temp               60
pulse                     24
respiratory_rate          58
temp_of_extremities       56
peripheral_pulse          69
mucous_membrane           47
capillary_refill_time     32
pain                      55
peristalsis               44
abdominal_distention      56
nasogastric_tube         104
nasogastric_reflux       106
nasogastric_reflux_ph    246
rectal_exam_feces        102
abdomen                  118
packed_cell_volume        29
total_protein             33
abdomo_appearance        165
abdomo_protein           198
outcome                    0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

In [90]:
target=horse[['outcome']]

In [91]:
target.value_counts()

outcome   
lived         178
died           77
euthanized     44
dtype: int64

In [92]:
features= horse.drop('outcome',axis=1)

In [93]:
features.shape

(299, 27)

In [94]:
features

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,distend_large,45.0,8.4,NaN,NaN,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,other,50.0,85.0,cloudy,2.0,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,normal,33.0,6.7,NaN,NaN,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,NaN,48.0,7.2,serosanguious,5.3,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,NaN,74.0,7.4,NaN,NaN,no,4300,0,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,yes,adult,533886,NaN,120.0,70.0,cold,NaN,pale_cyanotic,more_3_sec,...,distend_large,55.0,65.0,NaN,NaN,no,3205,0,0,no
295,no,adult,527702,37.2,72.0,24.0,cool,increased,pale_cyanotic,more_3_sec,...,distend_small,44.0,NaN,serosanguious,3.3,yes,2208,0,0,yes
296,yes,adult,529386,37.5,72.0,30.0,cold,reduced,pale_cyanotic,less_3_sec,...,distend_large,60.0,6.8,NaN,NaN,yes,3205,0,0,no
297,yes,adult,530612,36.5,100.0,24.0,cool,reduced,pale_pink,less_3_sec,...,distend_small,50.0,6.0,serosanguious,3.4,yes,2208,0,0,yes


In [95]:
features= pd.get_dummies(features)

In [96]:
features.shape

(299, 67)

In [97]:
features.columns

Index(['hospital_number', 'rectal_temp', 'pulse', 'respiratory_rate',
       'nasogastric_reflux_ph', 'packed_cell_volume', 'total_protein',
       'abdomo_protein', 'lesion_1', 'lesion_2', 'lesion_3', 'surgery_no',
       'surgery_yes', 'age_adult', 'age_young', 'temp_of_extremities_cold',
       'temp_of_extremities_cool', 'temp_of_extremities_normal',
       'temp_of_extremities_warm', 'peripheral_pulse_absent',
       'peripheral_pulse_increased', 'peripheral_pulse_normal',
       'peripheral_pulse_reduced', 'mucous_membrane_bright_pink',
       'mucous_membrane_bright_red', 'mucous_membrane_dark_cyanotic',
       'mucous_membrane_normal_pink', 'mucous_membrane_pale_cyanotic',
       'mucous_membrane_pale_pink', 'capillary_refill_time_3',
       'capillary_refill_time_less_3_sec', 'capillary_refill_time_more_3_sec',
       'pain_alert', 'pain_depressed', 'pain_extreme_pain', 'pain_mild_pain',
       'pain_severe_pain', 'peristalsis_absent', 'peristalsis_hypermotile',
       'perist

In [98]:
x_train,x_test,y_train,y_test= train_test_split(features,target,train_size=0.75,random_state=70)

In [99]:
x_train.isnull().sum()

hospital_number                      0
rectal_temp                         41
pulse                               16
respiratory_rate                    43
nasogastric_reflux_ph              184
                                  ... 
abdomo_appearance_serosanguious      0
surgical_lesion_no                   0
surgical_lesion_yes                  0
cp_data_no                           0
cp_data_yes                          0
Length: 67, dtype: int64

In [100]:
simple_impute= SimpleImputer(missing_values=np.nan,strategy='mean')

In [101]:
x_train= simple_impute.fit_transform(x_train)
x_test= simple_impute.fit_transform(x_test)

In [102]:
x_train

array([[5.28178e+05, 3.77000e+01, 6.60000e+01, ..., 1.00000e+00,
        0.00000e+00, 1.00000e+00],
       [5.34115e+05, 3.82000e+01, 4.80000e+01, ..., 1.00000e+00,
        1.00000e+00, 0.00000e+00],
       [5.28964e+05, 3.78000e+01, 3.00000e+01, ..., 1.00000e+00,
        1.00000e+00, 0.00000e+00],
       ...,
       [5.30624e+05, 4.00000e+01, 7.80000e+01, ..., 1.00000e+00,
        0.00000e+00, 1.00000e+00],
       [5.32349e+05, 3.84000e+01, 4.40000e+01, ..., 1.00000e+00,
        1.00000e+00, 0.00000e+00],
       [5.29135e+05, 3.78000e+01, 4.00000e+01, ..., 1.00000e+00,
        1.00000e+00, 0.00000e+00]])

In [103]:
a= pd.DataFrame(x_train)

In [104]:
a.isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
62    0
63    0
64    0
65    0
66    0
Length: 67, dtype: int64

In [105]:
dec_tree= DecisionTreeClassifier( criterion='entropy', splitter='random',max_depth=15)

In [106]:
dec_tree.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=15, splitter='random')

In [107]:
y_pred= dec_tree.predict(x_test)

In [108]:
y_pred

array(['lived', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived',
       'died', 'lived', 'euthanized', 'lived', 'euthanized', 'lived',
       'lived', 'lived', 'lived', 'lived', 'euthanized', 'lived', 'lived',
       'lived', 'lived', 'died', 'lived', 'died', 'died', 'died',
       'euthanized', 'died', 'lived', 'euthanized', 'lived', 'euthanized',
       'died', 'lived', 'euthanized', 'lived', 'lived', 'euthanized',
       'euthanized', 'lived', 'lived', 'lived', 'died', 'lived', 'lived',
       'euthanized', 'lived', 'died', 'lived', 'died', 'died',
       'euthanized', 'lived', 'euthanized', 'euthanized', 'euthanized',
       'euthanized', 'lived', 'lived', 'lived', 'lived', 'lived', 'lived',
       'died', 'lived', 'lived', 'lived', 'died', 'euthanized',
       'euthanized', 'died', 'euthanized', 'lived', 'died'], dtype=object)

In [109]:
accuracy_score(y_test,y_pred)

0.6533333333333333

In [110]:
confusion_matrix(y_test,y_pred)

array([[11,  3,  9],
       [ 0,  8,  3],
       [ 4,  7, 30]], dtype=int64)

In [111]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        died       0.73      0.48      0.58        23
  euthanized       0.44      0.73      0.55        11
       lived       0.71      0.73      0.72        41

    accuracy                           0.65        75
   macro avg       0.63      0.65      0.62        75
weighted avg       0.68      0.65      0.65        75



In [112]:
pd.DataFrame(y_pred).value_counts()

lived         42
euthanized    18
died          15
dtype: int64

In [113]:
params= {'criterion':['gini','entropy'], 'splitter':['best','random'],'max_depth':[1,2,5,7,8,10,12,15]}

In [114]:
grid_search= GridSearchCV(dec_tree,params)

In [115]:
grid_search.fit(x_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(criterion='entropy', max_depth=15,
                                              splitter='random'),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 5, 7, 8, 10, 12, 15],
                         'splitter': ['best', 'random']})

In [116]:
grid_search.best_params_

{'criterion': 'entropy', 'max_depth': 12, 'splitter': 'random'}